# This notebook shows how to use Orchestrator APIs for user experiments

## Configure the Environment

In [5]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']='pruth_0031379841'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [6]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib


In [7]:
try:
    print(f"{fablib.list_sites()}")
    #print(f"{fablib.get_available_resources().get_topology()}")

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()


Name      CPUs  Cores    RAM (G)    Disk (G)       Basic (100 Gbps NIC)    ConnectX-6 (100 Gbps x2 NIC)    ConnectX-5 (25 Gbps x2 NIC)    P4510 (NVMe 1TB)    Tesla T4 (GPU)    RTX6000 (GPU)
------  ------  -------  ---------  -------------  ----------------------  ------------------------------  -----------------------------  ------------------  ----------------  ---------------
TACC        10  316/320  2544/2560  116200/116400  635/635                 2/2                             4/4                            16/16               4/4               6/6
UTAH        10  318/320  2552/2560  116390/116400  635/635                 2/2                             3/4                            16/16               4/4               5/5
WASH         6  192/192  1536/1536  60600/60600    381/381                 2/2                             2/2                            10/10               2/2               3/3
MICH         6  180/192  1488/1536  60450/60600    375/381                 1/2

In [8]:
try:    
    for site_name in fablib.get_site_names():
        print(f"{fablib.show_site(site_name)}")

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()


----------------------------  --------------------------------
Name                          TACC
CPUs                          10
Cores ()                      316/320
RAM (G)                       2544/2560
Disk (G)                      116200/116400
Worker Count                  5
Physical Address              10100 Burnet Rd,Austin, TX 78758
Location Coordinates          (30.3899405, -97.7261806879021)
Basic (100 Gbps NIC)          635/635
ConnectX-6 (100 Gbps x2 NIC)  2/2
ConnectX-5 (25 Gbps x2 NIC)   4/4
P4510 (NVMe 1TB)              16/16
Tesla T4 (GPU)                4/4
RTX6000 (GPU)                 6/6
----------------------------  --------------------------------
----------------------------  -----------------------------------------------
Name                          UTAH
CPUs                          10
Cores ()                      318/320
RAM (G)                       2552/2560
Disk (G)                      116390/116400
Worker Count                  5
Physical Address 

## Create Slice
In Release 1.0, user is expected to assign the IP addresses manually. Please use the example comands indicated below:

## Configure Slice Parameters



In [9]:
slice_name = 'MySliceL2Net'
[site1,site2] = fablib.get_random_sites(count=2,avoid=[])
print(f"sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'
network1_name='net1'
network2_name='net2'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

sites: MAX, STAR


### Create the Slice

NIC component models options:
- NIC_Basic 
- NIC_ConnectX_5 
- NIC_ConnectX_6

In [10]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    [iface1a, iface1b] = node1.add_component(model='NIC_ConnectX_5', name=node1_nic_name).get_interfaces()
    
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    [iface2a, iface2b] = node2.add_component(model='NIC_ConnectX_5', name=node2_nic_name).get_interfaces()
    
    # Network
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1a], type='IPv4')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2a], type='IPv4')
    
     # Network
    net1 = slice.add_l2network(name='netL2', interfaces=[iface1b, iface2b])


    #Submit Slice Request
    slice.submit()
    #slice.submit(wait=False)
    #slice.wait_ssh(progress=True)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()


Exception: Failed to submit slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Mon, 14 Mar 2022 15:44:44 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '33', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Error', 'X-Error': 'Slice MySliceL2Net already exists'})
HTTP response body: Slice MySliceL2Net already exists



Traceback (most recent call last):
  File "/var/folders/_l/8dq3pgzx6bjgt8qrr1m9jgc80000gp/T/ipykernel_57974/2503453859.py", line 24, in <module>
    slice.submit()
  File "/usr/local/lib/python3.9/site-packages/fabrictestbed_extensions/fablib/slice.py", line 839, in submit
    raise Exception("Failed to submit slice: {}, {}".format(return_status, slice_reservations))
Exception: Failed to submit slice: Status.FAILURE, (500)
Reason: INTERNAL SERVER ERROR
HTTP response headers: HTTPHeaderDict({'Server': 'nginx/1.21.6', 'Date': 'Mon, 14 Mar 2022 15:44:44 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '33', 'Connection': 'keep-alive', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Headers': 'DNT, User-Agent, X-Requested-With, If-Modified-Since, Cache-Control, Content-Type, Range', 'Access-Control-Allow-Methods': 'GET, POST, PUT, DELETE, OPTIONS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Expose-Headers': 'Content-Length, Content-Range, X-Err

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    #slice.post_boot_config()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()


## Get the Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
    
    print(f"{slice.get_interface_map()}")
except Exception as e:
    print(f"Exception: {e}")


In [ ]:
import traceback
try:
    slice = fablib.get_slice(name=slice_name)
    #print(f"{slice.get_slivers()}")
    
    for network in slice.get_networks():
        print(f"{network.get_reservation_id()}")
        print(f"{network.get_gateway()}")
        print(f"{network.get_subnet()}")
        
        #s = slice.get_sliver(network.get_reservation_id())
        #s = network.get_sliver()
        #print(f"{s.sliver.gateway.gateway}")
        #print(f"{s.sliver.gateway.subnet}")

   #for node in slice.get_nodes():
   #     print(f"{node.get_reservation_id()}")
   #     print(f"{node.get_sliver()}")
   #     s = slice.get_sliver(node.get_reservation_id())
   #     print(f"{type(s.sliver)}")
   #     print(f"{s.sliver.management_ip}")
   #     for component in s.sliver.attached_components_info.list_devices():
   #         print(f"component: {component}")
   #     print(f"{s.sliver.attached_components_info}") 
        #self.allocation_constraints = None
        #self.image_type = None
        #self.image_ref = None
        #self.service_endpoint = None
        #self.network_service_info = None
        #self.site = None
        #self.location = None
        #self.boot_script = None


    #s = slice.get_sliver('8dd86f41-6401-4d29-b3d2-70846a3f73f1')
    #print(f"{type(s.sliver)}")
    #print(f"{s.sliver.gateway.gateway}")
    #print(f"{s.sliver.gateway.subnet}")
    #print(f"{s.sliver.site}")
    #print(f"{s.sliver.layer}")
    #for interface in s.sliver.interface_info:
    #    print(f"{type(interface)}")




    
    #for s in slice.get_slivers():
    #    if s.sliver_type == "NetworkServiceSliver": 
    #        print(f"{s}")
    #        print(f"{s.sliver.gateway.gateway}")
    #        print(f"{s.sliver.gateway.subnet}")
    #    #s.get_property(pname='reservation_info').error_message
    
    #for s in slice.get_slivers():
    #    print(f"{s}")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()


## Get the Nodes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Get the Interfaces

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
    
    #for iface in slice.get_interfaces():
    #    print(f"{iface}")
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Configure the interface on on  node1.  

```
ip addr add 192.168.1.10/24 dev eth1
```

In [ ]:
from ipaddress import IPv4Address, IPv6Address, IPv4Network, IPv6Network

addr = IPv4Address('192.168.0.1') + 1
print(f"{addr}")


#addr += 1
#print(f"{addr}")

iplist = list(IPv4Network('192.168.0.0/30'))
print(f"{ iplist }")
iplist_mod = iplist.remove(IPv4Address('192.168.0.0'))
print(f"{ iplist }")

network1 = IPv4Network('192.168.1.0/24')
network2 = IPv4Network('192.168.1.0/24')

print(f"{network1}")
print(f"{network2}")

print(f"{network1.prefixlen}")

#print(f"{network1+network2}")

In [ ]:


network1 = slice.get_network(name=network1_name)
print(f"{network1}")

network2 = slice.get_network(name=network2_name)
print(f"{network2}")


        

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_iface.ip_addr_add(addr=network1.get_gateway()+1, subnet=network1.get_subnet())
    
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    
    stdout, stderr = node1.execute(f'ip -6 addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node1.execute(f'ip -6 route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

### Configure Node2

Configure the interface on on  node2.  

```
ip addr add 192.168.1.11/24 dev eth1
```

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name)  
    node2_iface.ip_addr_add(addr=network2.get_gateway()+1, subnet=network2.get_subnet())
    
    node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    
    stdout, stderr = node2.execute(f'ip -6 addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node2.execute(f'ip -6 route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")